Training Dataset

In [10]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip

--2021-07-02 01:56:40--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 142.250.65.80, 142.250.188.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘rps.zip’

rps.zip             100%[===================>] 191.38M   299MB/s    in 0.6s    

2021-07-02 01:56:41 (299 MB/s) - ‘rps.zip’ saved [200682221/200682221]



Testing Dataset

In [11]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip

--2021-07-02 01:56:50--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.208, 142.250.73.240, 142.250.65.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘rps-test-set.zip’

rps-test-set.zip    100%[===================>]  28.15M  --.-KB/s    in 0.1s    

2021-07-02 01:56:50 (236 MB/s) - ‘rps-test-set.zip’ saved [29516758/29516758]



Validation Dataset

In [12]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-validation.zip

--2021-07-02 01:57:06--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-validation.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.81.208, 172.217.7.240, 172.217.13.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.81.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6623597 (6.3M) [application/zip]
Saving to: ‘rps-validation.zip’

rps-validation.zip  100%[===================>]   6.32M  --.-KB/s    in 0.04s   

2021-07-02 01:57:06 (170 MB/s) - ‘rps-validation.zip’ saved [6623597/6623597]



In [ ]:
!unzip /content/rps-test-set.zip

In [ ]:
!unzip /content/rps-validation.zip

In [ ]:
!unzip /content/rps.zip

Libraries

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

In [2]:
testgen = ImageDataGenerator(rescale=1/255.)

traingen = ImageDataGenerator(
    rescale=1/255.,
    horizontal_flip=True,
    rotation_range=90,
    vertical_flip=True,
    zoom_range=0.2,
    brightness_range=[0.2,1.0]
)

train_generator = traingen.flow_from_directory(
    '/content/rps',
    class_mode='categorical',
    batch_size=32,
    target_size=(200, 200)
)

test_generator = testgen.flow_from_directory(
    '/content/rps-test-set',
    class_mode='categorical',
    batch_size=32,
    target_size=(200, 200)
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


Model

In [3]:
model = Sequential([
    Conv2D(filters=16, kernel_size=3, padding='same', input_shape=(200, 200, 3), activation=tf.nn.relu),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu),
    MaxPooling2D(pool_size=(2, 2)),

    Dropout(rate=0.2),

    Flatten(),

    Dense(units=1000, activation=tf.nn.relu),
    Dropout(rate=0.25),

    Dense(units=3, activation=tf.nn.softmax)

])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 128)       7

In [5]:
from tensorflow.keras.optimizers import Adam

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [7]:
checkpoint = ModelCheckpoint("model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose = 1)

earlystop = EarlyStopping(monitor="val_loss",
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True)

callbacks = [earlystop, checkpoint]

In [8]:
model.fit(
    train_generator,
    epochs=100,
    validation_data=test_generator,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/100
79/79 [==============================] - 33s 378ms/step - loss: 1.1006 - accuracy: 0.3786 - val_loss: 1.0989 - val_accuracy: 0.5027

Epoch 00001: val_loss improved from inf to 1.09889, saving model to model.h5
Epoch 2/100
79/79 [==============================] - 29s 368ms/step - loss: 0.7697 - accuracy: 0.6623 - val_loss: 0.9480 - val_accuracy: 0.5349

Epoch 00002: val_loss improved from 1.09889 to 0.94797, saving model to model.h5
Epoch 3/100
79/79 [==============================] - 29s 369ms/step - loss: 0.4809 - accuracy: 0.8242 - val_loss: 0.7410 - val_accuracy: 0.6801

Epoch 00003: val_loss improved from 0.94797 to 0.74095, saving model to model.h5
Epoch 4/100
79/79 [==============================] - 29s 369ms/step - loss: 0.2830 - accuracy: 0.9020 - val_loss: 0.6838 - val_accuracy: 0.7312

Epoch 00004: val_loss improved from 0.74095 to 0.68380, saving model to model.h5
Epoch 5/100
79/79 [==============================] - 29s 369ms/step - loss: 0.2283 - accuracy: 0.926

In [12]:
train_generator.class_indices

{'paper': 0, 'rock': 1, 'scissors': 2}

In [22]:
import re
import glob

In [23]:
IMAGES = glob.glob('/content/*.png')
len(IMAGES)

33

In [26]:
def predict(image):
  stri = str(image).split('.')[0].split('/')[-1]
  result = re.sub('[0-9]+', ' ', stri)
  img = tf.keras.preprocessing.image.load_img(
    image, target_size=(200, 200)
  )

  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)

  prediction = model.predict(img_array)

  if prediction[0][0] == 1:
    print(f'Actual : {result}, Prediction : Paper')
  if prediction[0][1] == 1:
    print(f'Actual : {result}, Prediction : Rock')
  if prediction[0][2] == 1:
    print(f'Actual : {result}, Prediction : Scissors')

In [27]:
for image in IMAGES:
  predict(image)

Actual : scissors , Prediction : Scissors
Actual : paper , Prediction : Rock
Actual : rock , Prediction : Rock
Actual : rock , Prediction : Rock
Actual : scissors-hires , Prediction : Scissors
Actual : scissors , Prediction : Scissors
Actual : scissors , Prediction : Scissors
Actual : paper , Prediction : Paper
Actual : paper , Prediction : Paper
Actual : scissors , Prediction : Scissors
Actual : paper , Prediction : Scissors
Actual : rock , Prediction : Rock
Actual : scissors-hires , Prediction : Scissors
Actual : paper-hires , Prediction : Paper
Actual : rock-hires , Prediction : Rock
Actual : paper , Prediction : Rock
Actual : scissors , Prediction : Scissors
Actual : rock , Prediction : Rock
Actual : rock , Prediction : Rock
Actual : scissors , Prediction : Scissors
Actual : paper , Prediction : Paper
Actual : paper , Prediction : Paper
Actual : rock , Prediction : Rock
Actual : paper-hires , Prediction : Paper
Actual : scissors , Prediction : Scissors
Actual : rock , Prediction : 

In [29]:
import os


def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)


# Lets check the file size of MS Paint exe 
# or you can use any file path
file_path = "/content/model.h5"
print(file_size(file_path))

212.2 MB
